# Importar bibliotecas generales

In [84]:
from sys import maxsize #para imprimir arrays completos
import numpy as np
import pandas as pd
import re as regex
import math as math

# Configuración de opciones generales

In [85]:
#cantidad de elementos que se muestran cuando se imprimen resultados
np.set_printoptions(threshold=maxsize)

pd.set_option('display.max_columns',1000000)

# Importar bibliotecas propias 

In [86]:
#biblioteca para completar valores Nan de la columna rooms
%run "analize_description_title.ipynb"

# Leer el data set desde el archivo y generar el data frame

In [87]:
pathArchivoDataSet = 'properatti.csv'
df = pd.read_csv(pathArchivoDataSet)

# Definimos datos generales que se van a usar en la notebook

In [88]:
#definimos un diccionario con valores de conversion entre monedas locales y el USD
conversion_USD_a_monedas_locales = { 'ARS': 63, 'PEN': 3.53, 'UYU': 43.41, 'USD': 1}

# Detalle generales del data frame

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

In [90]:
df.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mataderos_lavadero_patio_inmobiliaria-fedele,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PATIO GRANDE, CON LAVADERO, LIVING COMEDOR CON COCINA INTEGRADA, ARTEFACTO DE COCINA, ALACENA, BAÑO COMPLETO, Y DORMITORIO. TODO EN EXCELENTE ESTADO, PARA HABITAR.NO ES APTO CREDITO Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIKufMJMwHtquzhk=/trim/198x0/smart/filters:strip_icc()/inmobiliariafedele.com/photos/CARHUE%202278-IMG_3400.JPG
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_departamentos_la-plata_balcon_lavadero_toilette_garage_estrenar_antonini-propiedades_dcp,"Venta de departamento en décimo piso al frente, a estrenar. Living comedor con ventanales hacia el balcón con pisos de madera. Cocina completa con doble mesada, muebles bajo mesada. Lavadero separado. Toilette.Dormitorio con placard. Segundo dormitorio a dividir. Calefacción por radiadorBaño completo.Balcón corrido. Posibilidad de cochera semicubierta. U$D 20.000 Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv45KAKirGL3JM8=/trim/198x0/smart/filters:strip_icc()/antoninipropiedades.com/photos/401-1-7%20y%2037%2010A%201.jpg
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_departamentos_mataderos_lavadero_placard_inmobiliaria-fedele,"2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON LAVADERO, BAÑO COMPLETO Y DORMITORIO CON PLACARD. BAJAS EXPENSAS, EXCELENTE UBICACION Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_Ejwfu6QpW4cH0=/trim/198x0/smart/filters:strip_icc()/inmobiliariafedele.com/photos/GUAMINI%201171-IMG_3403.JPG
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_liniers_patio_g-goffredo,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso.Reciclado a nuevo. ""Muy buen estado"" Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SPyleyeQlIypI8w=/trim/198x0/smart/filters:strip_icc()/ggoffredo.com/photos/GG0040-1563480102.jpg
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_departamentos_centro_cristina-pavone-negocios-inmobiliarios,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFRENTE, EDIFIC

In [91]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

# La primer columna no tiene un nombre asignado

In [92]:
type(df.iloc[0,0])

numpy.int64

In [93]:
len(df.loc[df.index == df['Unnamed: 0']])

121220

## Para todas las filas del data frame, el valor de la primer columna es igual al valor del index. Se puede asumir que dicha columna corresponde a un campo ID. Ya que contamos con el index del data frame, borramos la columna ID

In [94]:
df.rename(columns={'Unnamed: 0': 'Id'}, inplace = True)
df.columns

Index(['Id', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

# Limpieza del data set 

### Removemos filas duplicadas

In [95]:
print('Total de filas del data frame: ' + str(len(df)))

Total de filas del data frame: 121220


In [96]:
#eliminamos duplicados verificando las siguientes columnas
df.drop_duplicates(subset=['operation', 'property_type','place_name','place_with_parent_names', \
                          'country_name', 'state_name', 'lat', 'lon', 'price', 'currency', 'price_aprox_local_currency', \
                          'price_aprox_usd','surface_total_in_m2', 'surface_covered_in_m2',\
                          'price_usd_per_m2','price_per_m2','floor','expenses', 'description', 'title'],keep='last', inplace=True)

In [97]:
print('Total de filas del data frame sin duplicados: ' + str(len(df)))

Total de filas del data frame sin duplicados: 116068


In [98]:
#reseteamos los índices para poder seguir usándolos sin problemas
df = df.reset_index();

### Se eliminan columnas innecesarias

In [99]:
df[~df['floor'].isnull() & df['floor'] > 20]

,index,Id,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail


In [100]:
#quitamos la columna floor, ya que es difícil de inferir a partir de las demás columnas, por simplicidad se quita de los datos
df = df.drop(labels='floor', axis = 1)

In [101]:
#quitamos la columna image_thumbnail ya que no aporta información relevante
df = df.drop(labels='image_thumbnail', axis= 1)

In [102]:
#quitamos la columna properati_url ya que no aporta información relevante
df = df.drop(labels='properati_url', axis= 1)

In [103]:
#chequeamos las columnas que nos quedan
df.columns

Index(['index', 'Id', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms',
       'expenses', 'description', 'title'],
      dtype='object')

### Se eliminan datos (filas) que no pudieron ser completados

In [104]:
#se eliminan filas cuyo campo description está vacío, dicha columna se tomó como fuente para otras columnas
#al venir vacía otros datos no pueden ser completados
emptyDescriptionIndexes = df[df['description'].isnull()].index

#borramos las filas con los correspondientes indices
df = df.drop(emptyDescriptionIndexes)

#reseteamos los índices para poder seguir usándolos sin problemas
df = df.reset_index();

# Completar valores faltantes

### A partir del dato geonames_id, generamos tres columnas nuevas, que corresponden a la coordenada representativa de cada zona (representada por un geonames_id). Esta coordenada no corresponde a la de la propiedad

##### Leemos directamente el archivo CSV generado en la notebook auxiliar "Completar coordenadas desde geonames_id" . Para ver el proceso remitirse a dicha notebook

In [105]:
latLngCSVFileName = 'latLngFromGeonames.csv'
#cargamos el archivo
latLongDF = pd.read_csv(latLngCSVFileName)

In [106]:
#mergeamos el data frame leído con el da properatti usando el dato geonames_id
df = df.merge(latLongDF, how='left', left_on='geonames_id', right_on='geonames_id', suffixes=('', '_geonames'))

### Datos de localidades: place_with_parent_names, place_name, country_name, state_name

In [107]:
df[['place_with_parent_names', 'place_name','country_name','state_name']].isnull().sum()

place_with_parent_names    0 
place_name                 23
country_name               0 
state_name                 0 
dtype: int64

##### Solo faltan 23 valores en la columna 'place_name' (información de barrio, zona, ciudad, etc.) que se intentarán obtener del la columna 'place_with_parent_names'.

In [108]:
#seteamos la longitud del output para mejor lectura
pd.set_option('display.max_colwidth', -1)

##### Tomamos las columnas necesarias para completar los faltantes

In [109]:
df[['place_with_parent_names', 'place_name','country_name','state_name']]

,place_with_parent_names,place_name,country_name,state_name
0,|Argentina|Capital Federal|Mataderos|,Mataderos,Argentina,Capital Federal
1,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,La Plata,Argentina,Bs.As. G.B.A. Zona Sur
2,|Argentina|Capital Federal|Mataderos|,Mataderos,Argentina,Capital Federal
3,|Argentina|Capital Federal|Liniers|,Liniers,Argentina,Capital Federal
4,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Centro,Argentina,Buenos Aires Costa Atlántica
...,...,...,...,...
116061,|Argentina|Capital Federal|Belgrano|,Belgrano,Argentina,Capital Federal
116062,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|Beccar|,Beccar,Argentina,Bs.As. G.B.A. Zona Norte
116063,|Argentina|Capital Federal|Villa Urquiza|,Villa Urquiza,Argentina,Capital Federal
116064,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Plaza Colón|,Plaza Colón,Argentina,Buenos Aires Costa Atlántica


##### Se crea una nueva columna con los valores de la columna 'place_with_parent_names' en forma de lista

In [110]:
def disgrega(valor):
    return valor.strip('|').split('|')

df['lista_auxiliar'] = df['place_with_parent_names'].apply(lambda x: disgrega(x))

In [111]:
df['lista_auxiliar']

0         [Argentina, Capital Federal, Mataderos]                              
1         [Argentina, Bs.As. G.B.A. Zona Sur, La Plata]                        
2         [Argentina, Capital Federal, Mataderos]                              
3         [Argentina, Capital Federal, Liniers]                                
4         [Argentina, Buenos Aires Costa Atlántica, Mar del Plata, Centro]     
                                        ...                                    
116061    [Argentina, Capital Federal, Belgrano]                               
116062    [Argentina, Bs.As. G.B.A. Zona Norte, San Isidro, Beccar]            
116063    [Argentina, Capital Federal, Villa Urquiza]                          
116064    [Argentina, Buenos Aires Costa Atlántica, Mar del Plata, Plaza Colón]
116065    [Argentina, Capital Federal]                                         
Name: lista_auxiliar, Length: 116066, dtype: object

##### Se calcula cuantos elementos tiene cada lista

In [112]:
df['conteo'] = df['lista_auxiliar'].apply(lambda x: len(x))
df['conteo']

0         3
1         3
2         3
3         3
4         4
         ..
116061    3
116062    4
116063    3
116064    4
116065    2
Name: conteo, Length: 116066, dtype: int64

##### Se determina cual es el mayor y menor números de elemntos en un registro de 'place_with_parent_names'

In [113]:
df['conteo'].max()

5

In [114]:
df['conteo'].min()

2

##### Se necesitarán 5 columnas como máximo para desempacar las distintas jerarquías de las locaciones. Mediante el comando apply se obtienen 5 columnas extras con información de localización por regiones.

In [115]:
def chequeo2(valor):
    if len(valor) > 2:
        return valor[2]
    
    
def chequeo3(valor):
    if len(valor) > 3:
        return valor[3]
    
def chequeo4(valor):
    if len(valor) > 4:
        return valor[4]

df['loc1'] = df['lista_auxiliar'].apply(lambda x: x[0])
df['loc2'] = df['lista_auxiliar'].apply(lambda x: x[1])
df['loc3'] = df['lista_auxiliar'].apply(lambda x: chequeo2(x))
df['loc4'] = df['lista_auxiliar'].apply(lambda x: chequeo3(x))
df['loc5'] = df['lista_auxiliar'].apply(lambda x: chequeo4(x))

In [116]:
daux = df[df['loc5'].notnull()]
daux[['loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

,loc1,loc2,loc3,loc4,loc5
17,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio El Golf
33,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio El Golf
269,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Las Glorietas
400,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Los Lagos
430,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Las Glorietas
...,...,...,...,...,...
112307,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,QBay Yacht
113366,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Los Lagos
113894,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Barrancas del Lago
114594,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Barrancas del Lago


##### Verificamos que los registros que tenían place_name vacíos tengan valores válidos en las nuevas columnas para cubrir el faltante

In [117]:
len(df[df['place_name'].isna() & ~df['loc5'].isna()])

0

In [118]:
df[df['place_name'].isna()][['lat','lon','geonames_id','country_name', 'state_name','place_name','place_with_parent_names', 'loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

,lat,lon,geonames_id,country_name,state_name,place_name,place_with_parent_names,loc1,loc2,loc3,loc4,loc5
6085,-34.400897,-58.638098,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
9741,-34.400704,-58.638067,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
10958,-34.402688,-58.626261,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
14318,-34.400799,-58.638174,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
18038,-34.403278,-58.626350,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
21286,-34.399704,-58.638405,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
23018,-34.400516,-58.641800,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
24059,-34.400729,-58.637778,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
37930,-34.425087,-58.579659,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
44891,-34.396074,-58.637380,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None


##### Vemos que a las fila que les falta el place_name corresponden todas al municipio de Tigre

In [119]:
#completamos los que tienen Nan en place_name con el valor Tigre
df.loc[df['place_name'].isnull(),'place_name'] = 'Tigre'

In [120]:
len(df[df['place_name'].isnull()])

0

##### Eliminamos columnas auxiliares generadas

In [121]:
df.drop(labels=['loc1', 'loc2','loc3','loc4','loc5', 'conteo','lista_auxiliar'], axis = 1, inplace = True)

##### Estandarizaramos los valores de localizaciones

In [122]:
#los datos de países ya están estandarizados
list(df['country_name'].unique())

['Argentina']

In [123]:
#hay registros que tienen en place_name nombres de zonas del conourbano bonaerense
df[df['place_name'] == 'Bs.As. G.B.A. Zona Norte']

,level_0,index,Id,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,expenses,description,title,lat_geonames,lat-lon_geonames,lon_geonames,place_name_geonames
1734,1734,1856,1856,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0,70.0,NaN,NaN,NaN,NaN,"Casa 500m², Argentina, Bs.As. G.B.A. Zona Norte, por U$S 50.000",U$D 50.000 - Casa en Venta - S/n S/N,-36.0,"-36,-60",-60.0,Buenos Aires
2176,2176,2448,2448,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.0,150.0,NaN,NaN,3.0,NaN,"CASA ESTILO ALPINA, DE TRES PLANTAS EN BARRIO LOS CARDALES COUNTRY CLUB . COCINA LIVING COMEDOR, TRES DORMITORIOS DOS BAÑOS . COCHERA. GALERIA PARRILLA LAVADERO. CASA SOBREUN SOLO LOTE DE 750 MTS EXPENSAS $ 7500 PESOS.",U$D 175.000 - Casa en Venta -,-36.0,"-36,-60",-60.0,Buenos Aires
3559,3559,3919,3919,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.0,150.0,NaN,NaN,3.0,NaN,"CASA ESTILO ALPINA, DE TRES PLANTAS EN BARRIO LOS CARDALES COUNTRY CLUB . COCINA LIVING COMEDOR, TRES DORMITORIOS DOS BAÑOS . COCHERA. GALERIA PARRILLA LAVADERO. CASA SOBRE DOS LOTES DE 750 MTS CADA UNO.EXPENSAS $ 14000 PESOS.",U$D 190.000 - Casa en Venta -,-36.0,"-36,-60",-60.0,Buenos Aires
3877,3877,4239,4239,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2400.0,170.0,NaN,NaN,4.0,NaN,"Casa en barrrio cerrado en ""El Solar de Capilla"", lote de 40 por 60, casa tipo country con arboles frutales, muy buena sombra, pileta con filtro nuevo, galeria,con kichinette, agua fria caliente, parrilla, ventiladores de techo en galeria de 10 por 4. 3 dormitorios, 2 baños, gran living comedor,con hogar a leña, pasaplatos y cocina,gas, termo,direct tv,alarma,.Lote central, club house, canchas de tenis,2 piletas grandes y chicos,muy buena entrada desde la ruta,bajas expensas,vecinos q viven todo el año.-lugar muy agreste,muy LLEGAR:Por la autopista Panamericana, ramal Pilar hasta el cruce con la ruta 8. Se toma esta ruta hacia la derecha hasta el Km 69. Allí se debe girar a la derecha 700 metros y luego volver a girar a la derecha. El acceso se encuentra a 200 metros.La venta de este inmueble está sujeta a la tramitación del Código de Transferencia de Inmuebles (COTI), de conformidad con la normativa vigente (Res. AFIP 2371/08, 2439/08 y ccs.) por parte del propietario.",U$D 220.000 - Casa en Venta -,-36.0,"-36,-60",-60.0,Buenos Aires
5036,5036,5426,5426,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,235000.0,USD,4146457.5,235000.0,200.0,200.0,1175.000000,1175.000000,NaN,NaN,"CASA CUATRO AMBIENTES A LA VENTA EN CARDALESBARRIO ""CARDALES VILLAGE"" LOTE Nº 7CASA CUATRO AMBIENTES DE 200 MT2 CUBIERTOS SOBRE UN LOTE DE 600 MTS, EN DOS PLANTA AL FRENTE; CON RECEPCIÓN, LIVING AMPLIO, TOILETTE, COCINA SEPARADA CON AMOBLAMIENTO DE BAJO MESADA Y ALACENA, TRES DORMITORIOS (DORMITORIO PRINCIPAL EN SUITE), TRES BAÑOS COMPLETOS, PARQUE Y PILETA. VALOR U$S 235.000.- - - FEDERICO NEGRO ESTUDIO INMOBILIARIO",CASA EN VENTA,-36.0,"-36,-60",-60.0,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112918,112920,117729,117729,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,580000.0,USD,10233810.0,580000.0,423.0,418.0,1371.158392,1387.559809,NaN,NaN,"Excelente Casa en venta en Campana Centro

In [124]:
#correjimos esos valores tomando los nombres de los partidos obtenidos desde el archivo de lat y long de geonames
df.loc[df['place_name'] == 'Bs.As. G.B.A. Zona Norte', 'place_name'] = df['place_name_geonames']
df.loc[df['place_name'] == 'Bs.As. G.B.A. Zona Oeste', 'place_name'] = df['place_name_geonames']
df.loc[df['place_name'] == 'Bs.As. G.B.A. Zona Sur', 'place_name'] = df['place_name_geonames']

In [125]:
print('Cantidad place_name con Bs.As. G.B.A. Zona Norte: ' + str(len(df[df['place_name'] == 'Bs.As. G.B.A. Zona Norte'])))
print('Cantidad place_name con Bs.As. G.B.A. Zona Oeste: ' + str(len(df[df['place_name'] == 'Bs.As. G.B.A. Zona Oeste'])))
print('Cantidad place_name con Bs.As. G.B.A. Zona Sur: ' + str(len(df[df['place_name'] == 'Bs.As. G.B.A. Zona Sur'])))
print('Cantidad place_name con Buenos Aires Costa Atlántica: ' + str(len(df[df['place_name'] == 'Buenos Aires Costa Atlántica'])))
print('Cantidad place_name con Buenos Aires Interior: ' + str(len(df[df['place_name'] == 'Buenos Aires Interior'])))

Cantidad place_name con Bs.As. G.B.A. Zona Norte: 0
Cantidad place_name con Bs.As. G.B.A. Zona Oeste: 0
Cantidad place_name con Bs.As. G.B.A. Zona Sur: 0
Cantidad place_name con Buenos Aires Costa Atlántica: 27
Cantidad place_name con Buenos Aires Interior: 106


In [126]:
#los datos de state_name ya están estandarizados
zonas = list(df['state_name'].unique())
zonas.sort()
zonas

['Bs.As. G.B.A. Zona Norte',
 'Bs.As. G.B.A. Zona Oeste',
 'Bs.As. G.B.A. Zona Sur',
 'Buenos Aires Costa Atlántica',
 'Buenos Aires Interior',
 'Capital Federal',
 'Catamarca',
 'Chaco',
 'Chubut',
 'Corrientes',
 'Córdoba',
 'Entre Ríos',
 'Formosa',
 'Jujuy',
 'La Pampa',
 'La Rioja',
 'Mendoza',
 'Misiones',
 'Neuquén',
 'Río Negro',
 'Salta',
 'San Juan',
 'San Luis',
 'Santa Cruz',
 'Santa Fe',
 'Santiago Del Estero',
 'Tierra Del Fuego',
 'Tucumán']

In [127]:
#los datos de state_name ya están estandarizados
zonas = list(df['place_name'].unique())
zonas.sort()
zonas

[' Country Maschwitz Club',
 ' los alamos',
 '9 de Abril',
 'Abasto',
 'Abril Club de Campo',
 'Acacias Blancas',
 'Acassuso',
 'Achiras',
 'Adolfo Alsina',
 'Adrogué',
 'Aeropuerto Internacional Ezeiza',
 'Agronomía',
 'Agua Blanca',
 'Agua de Oro',
 'Aguas Verdes',
 'Albanueva Barrio Cerrado',
 'Aldea Brasilera',
 'Aldea Salto',
 'Aldo Bonzi',
 'Alejandro Korn',
 'Alejandro Roca',
 'Alem',
 'Alfar',
 'Almafuerte',
 'Almagro',
 'Almirante Brown',
 'Alta Gracia',
 'Altamira',
 'Alto Los Cardales',
 'Altos de Hudson I',
 'Altos de Hudson II',
 'Altos de Manzanares 1 y 2',
 'Altos de Matheu',
 'Altos del Golf',
 'Altos del Pilar',
 'Aluminé',
 'Alvear',
 'Anisacate',
 'Apóstoles',
 'Arequito',
 'Armenia Country Club',
 'Arocena',
 'Arroyo Ceibal',
 'Arroyo Leyes',
 'Arroyo Seco',
 'Arturo Seguí',
 'Ascochinga',
 'Ataliva',
 'Avellaneda',
 'Azara',
 'Azul',
 'Bahía Blanca',
 'Bahía del Sol',
 'Balcarce',
 'Balnearia',
 'Balneario San Cayetano',
 'Balvanera',
 'Banda del Río Salí',
 'Banfi

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116066 entries, 0 to 116065
Data columns (total 29 columns):
level_0                       116066 non-null int64
index                         116066 non-null int64
Id                            116066 non-null int64
operation                     116066 non-null object
property_type                 116066 non-null object
place_name                    116066 non-null object
place_with_parent_names       116066 non-null object
country_name                  116066 non-null object
state_name                    116066 non-null object
geonames_id                   97893 non-null float64
lat-lon                       67784 non-null object
lat                           67784 non-null float64
lon                           67784 non-null float64
price                         98529 non-null float64
currency                      98528 non-null object
price_aprox_local_currency    98529 non-null float64
price_aprox_usd               98529 non-null f

In [129]:
#estandarizamos nombres de barrios de CABA (tomado del trabajo del grupo 2)
reemplazo_nombres = {'Abasto' : 'Balvanera',
                     'Barrio Norte' : 'Recoleta',
                     'Catalinas': 'Retiro',
                     'Congreso' : 'Balvanera',
                     'Las Cañitas' : 'Palermo',
                     'Once' : 'Balvanera',
                     'Pompeya' : 'Nueva Pompeya',
                     'Tribunales' : 'San Nicolás',
                     'Centro / Microcentro' : 'San Nicolás'}


df = df.replace({'place_name': reemplazo_nombres})      

# Datos de coordenadas de propiedades

#### Armamos 2 DataFrame:
    (A) 1 DataFrame para Ordenada Latitud: de columnas 'place_name' y 'lat'
    (B) 1 DataFrame para Ordenada Longitud: de columnas 'place_name' y 'lon'

(A) DF de Ordenada Latitud

In [130]:
# Definimos el subset del dataframe 'data' para quedarnos sólo con las columnas 'place_name' y 'lat'
# una vez definida se pasa el método ".head()" que despliega las primeras 5 filas del Dataframe 

dflatitud=df[['place_name', 'lat']]
dflatitud.head()

,place_name,lat
0,Mataderos,-34.661824
1,La Plata,-34.903883
2,Mataderos,-34.652262
3,Liniers,-34.647797
4,Centro,-38.002626


Comprobamos cantidad de filas completas de la columna 'lat' en comparación al total de filas

In [131]:
dflatitud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116066 entries, 0 to 116065
Data columns (total 2 columns):
place_name    116066 non-null object
lat           67784 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.7+ MB


De (A) nos quedamos solo con los campos con datos, de la columna 'lat', desechando los valores 'NaN'

In [132]:
dflatitud_1=dflatitud.dropna()
dflatitud_1.head()

,place_name,lat
0,Mataderos,-34.661824
1,La Plata,-34.903883
2,Mataderos,-34.652262
3,Liniers,-34.647797
4,Centro,-38.002626


Volvemos a tomar cantidad de datos por fila y vemos que se eliminaron los NaN del Dataframe.
Observamos también, que en 'lat' bajaron 23 filas más, lo que indica que en 'place_name' también eran NaN. Se eliminan son sólo 23 en 121220 (0,018%)

In [133]:
dflatitud_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67784 entries, 0 to 116063
Data columns (total 2 columns):
place_name    67784 non-null object
lat           67784 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.6+ MB


Calulamos la media de las latitudes por grupo de localidad, con el método ".groupby()" P.ej.: toma todas las filas de localidad igual a "Abasto", de la columna 'place_name' y calcula la media de las Latitudes

In [134]:
dflatitud_1=dflatitud_1.groupby(['place_name']).mean()
dflatitud_1.head()

,lat
place_name,
Country Maschwitz Club,-34.377707
los alamos,-34.423989
9 de Abril,-34.756258
Abril Club de Campo,-34.814314
Acacias Blancas,-34.405774


Comprobamos que la cantidad de filas vuelve a disminuir, puesto que sólo muestra la media de localidades únicas.

In [135]:
len(dflatitud_1)

949

(B) DF de Ordenada Longitud

###### Armo un subset sólo con las columnas 'place_name' y 'lon'

In [136]:
# Definimos el subset del dataframe 'data' para quedarnos sólo con las columnas 'place_name' y 'lat'
# una vez definida se pasa el método ".head()" que despliega las primeras 5 filas del Dataframe 
dflongitud=df[['place_name', 'lon']]
dflongitud.head()

,place_name,lon
0,Mataderos,-58.508839
1,La Plata,-57.964330
2,Mataderos,-58.522982
3,Liniers,-58.516424
4,Centro,-57.549447


Comprobamos cantidad de filas completas de la columna 'lon' en comparación al total de filas

In [137]:
dflongitud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116066 entries, 0 to 116065
Data columns (total 2 columns):
place_name    116066 non-null object
lon           67784 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.7+ MB


De (B) nos quedamos solo con los campos con datos, de la columna 'lon', desechando los valores 'NaN'

In [138]:
dflongitud_1=dflongitud.dropna()
dflongitud_1.head()

,place_name,lon
0,Mataderos,-58.508839
1,La Plata,-57.964330
2,Mataderos,-58.522982
3,Liniers,-58.516424
4,Centro,-57.549447


Volvemos a tomar cantidad de datos por fila y vemos que se eliminaron los NaN del Dataframe.
Observamos también, que en 'lon' bajaron 23 filas más, lo que indica que en 'place_name' también eran NaN. Se eliminan son sólo 23 en 121220 (0,018%)

In [139]:
dflongitud_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67784 entries, 0 to 116063
Data columns (total 2 columns):
place_name    67784 non-null object
lon           67784 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.6+ MB


Calulamos la media de las longtudes por grupo de localidad, con el método ".groupby()" P.ej.: toma todas las filas de localidad igual a "Abasto", de la columna 'place_name' y calcula la media de las Longitudes

In [140]:
dflongitud_1=dflongitud_1.groupby(['place_name']).mean()
dflongitud_1.head()

,lon
place_name,
Country Maschwitz Club,-58.755411
los alamos,-58.601876
9 de Abril,-58.498288
Abril Club de Campo,-58.156066
Acacias Blancas,-58.760994


Comprobamos que la cantidad de filas vuelve a disminuir, puesto que sólo muestra la media de localidades únicas.

In [141]:
len(dflongitud_1)

949

Definimos un nuevo dataframe que una las columnas lat y lon de los puntos (A) y (B).
Para ello vamos utilizar el método ".merge()" de Pandas

In [142]:
dflatlon=pd.merge(dflatitud_1, dflongitud_1, left_on='place_name', right_on='place_name')
dflatlon

,lat,lon
place_name,,
Country Maschwitz Club,-34.377707,-58.755411
los alamos,-34.423989,-58.601876
9 de Abril,-34.756258,-58.498288
Abril Club de Campo,-34.814314,-58.156066
Acacias Blancas,-34.405774,-58.760994
...,...,...
William Morris,-34.581910,-58.655338
Yerba Buena,-26.813276,-65.295010
Zelaya,-35.053791,-59.660925


Tomamos al nuevo dataframe y generamos un nuevo índice para que 'place_name' vuelva a ser una variable del df

In [143]:
dflatlon.reset_index(inplace=True)
dflatlon

,place_name,lat,lon
0,Country Maschwitz Club,-34.377707,-58.755411
1,los alamos,-34.423989,-58.601876
2,9 de Abril,-34.756258,-58.498288
3,Abril Club de Campo,-34.814314,-58.156066
4,Acacias Blancas,-34.405774,-58.760994
...,...,...,...
944,William Morris,-34.581910,-58.655338
945,Yerba Buena,-26.813276,-65.295010
946,Zelaya,-35.053791,-59.660925
947,Zárate,-34.096436,-59.067763


Tomamos al dataframe original (df) y hacemos un Join con Pandas, sumando al final del DF las columnas con las medias de 'lat' y 'lon'

In [144]:
df = df.merge(dflatlon, how='left', left_on='place_name', right_on='place_name', suffixes=('','_mediaPorZona'))
df.head(5)

,level_0,index,Id,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,expenses,description,title,lat_geonames,lat-lon_geonames,lon_geonames,place_name_geonames,lat_mediaPorZona,lon_mediaPorZona
0,0,0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PATIO GRANDE, CON LAVADERO, LIVING COMEDOR CON COCINA INTEGRADA, ARTEFACTO DE COCINA, ALACENA, BAÑO COMPLETO, Y DORMITORIO. TODO EN EXCELENTE ESTADO, PARA HABITAR.NO ES APTO CREDITO Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB TIPO CASA SIN EXPENSAS EN PB,-34.666670,"-34.66667,-58.5",-58.500000,Mataderos,-34.666326,-58.519094
1,1,1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,"Venta de departamento en décimo piso al frente, a estrenar. Living comedor con ventanales hacia el balcón con pisos de madera. Cocina completa con doble mesada, muebles bajo mesada. Lavadero separado. Toilette.Dormitorio con placard. Segundo dormitorio a dividir. Calefacción por radiadorBaño completo.Balcón corrido. Posibilidad de cochera semicubierta. U$D 20.000 Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37,-35.000000,"-35,-58",-58.000000,Partido de La Plata,-34.924277,-57.965973
2,2,2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,"2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON LAVADERO, BAÑO COMPLETO Y DORMITORIO CON PLACARD. BAJAS EXPENSAS, EXCELENTE UBICACION Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,-34.666670,"-34.66667,-58.5",-58.500000,Mataderos,-34.666326,-58.519094
3,3,3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso.Reciclado a nuevo. ""Muy buen estado"" Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",PH 3 amb. cfte. reciclado,-34.642860,"-34.64286,-58.52386",-58.523860,Liniers,-34.643508,-58.516984
4,4,4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFRENTE, EDIFICIO DE POCAS UNIDADES Y MUY BIEN CONSERVADO. EN ESTADO ORIGINAL.-POSEE: 1 DORMITORIOS, 1 BAÑO COMPLETO, LIVING COMEDOR Y COCINA SEPARADA.-EN ESTADO ORIGINAL.- Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA MITRE,-38.002626,"-38.0026256,-57.5494468",-57.549447,NaN,-38.000247,-57.547260


##### Property_Type => OK

In [145]:
#Vemos los tipos de propiedades existentes, y su correspondiente cantidad

df['property_type'].value_counts()

apartment    66572
house        39758
PH           5664 
store        4072 
Name: property_type, dtype: int64

In [146]:
#Revisamos si alguna linea tiene el tipo de propiedad vacio
#Vemos que no, que todas las 121.220 lineas existentes tienen el campo completo

df['property_type'].value_counts().sum()

116066

In [147]:
house = df.loc[:, 'property_type'] == 'house'
print("La cantidad de house es: ",house.sum())

apartment = df.loc[:, 'property_type'] == 'apartment'
print("La cantidad de apartment es: ",apartment.sum())

PH = df.loc[:, 'property_type'] == 'PH'
print("La cantidad de PH es: ",PH.sum())

store = df.loc[:, 'property_type'] == 'store'
print("La cantidad de store es: ",store.sum())

tipodepropiedad = house & apartment & PH & store
print("La cantidad de propiedades sin tipo es: ",tipodepropiedad.sum())

La cantidad de house es:  39758
La cantidad de apartment es:  66572
La cantidad de PH es:  5664
La cantidad de store es:  4072
La cantidad de propiedades sin tipo es:  0


##### Surface (covered y total)

###### Opcion 1: funcion para obtener metros cuadrados (a partir de surface_total_in_m2 y surface_covered_in_m2)

In [148]:
print('Cantidad registros surface_covered_in_m2 sin valor: ' + str(df['surface_covered_in_m2'].isnull().sum()))
print('Cantidad registros surface_total_in_m2 sin valor: ' + str(df['surface_total_in_m2'].isnull().sum()))

Cantidad registros surface_covered_in_m2 sin valor: 18837
Cantidad registros surface_total_in_m2 sin valor: 38360


###### Vamos a tratar de llenar los datos faltantes de superficie total en el dataset, rellenando con el promedio por barrio de cada publicacion

In [149]:
#Primero vamos a ordenar el dataset para poder tener los partidos en orden y limpios
df.rename(index=str, inplace=True)

In [150]:
#Primero hacmemos data wrangling de la columna "place_with_parent_names"

df.place_with_parent_names = df.place_with_parent_names.map(str.lower) #llevo a minusculas para evitar duplicados

grouped_places = df.groupby(['place_with_parent_names']) #agrupo por place_with_parent_names

dictio_places = grouped_places.groups.keys() #genero diccionario de places

cantidad_places = len(dictio_places) #cuento la cantidad de places distintos

print("Cantidad de place_with_parent_names distintos: ",cantidad_places)

Cantidad de place_with_parent_names distintos:  1164


In [151]:
count_x_places = grouped_places.agg({"Id": "count"}) #agrupo y cuento

count_x_places = count_x_places.rename(index=str, columns={"Id": "cantidad"}) #renombro la columna por cantidad

count_x_places = count_x_places.sort_values(by=['cantidad'], ascending=False) #ordeno por cantidad descendente

In [152]:
len(count_x_places.query("cantidad > 50")) # places con mas de <n> registros. Solo consulta

258

In [153]:
list_places = [sub_places.split('|') for sub_places in count_x_places.index]

df_places = pd.DataFrame(list_places, 
                         index = count_x_places.index, 
                         columns =['none1','pais','provincia','partido','localidad','barrio','none2'])

df_places = df_places.drop(['none1', 'none2'], axis=1) # elimino none1 y none2
df_places = df_places.drop(['pais'], axis=1) # elimino pais ya que todos son argentina
df_places[df_places.barrio.notnull()] # el unico que tiene barrios es tigre-nordelta
df_places = df_places.drop(['barrio'], axis=1) # elimino barrio tambien

In [154]:
# Para cada columna busco vacios y asigno None

for column_name in df_places.columns:
    df_places[column_name][df_places[column_name].apply(lambda column_name: True if regex.search('^\s*$', str(column_name)) else False)]=None

df_places = df_places.sort_values(by=['provincia','partido','localidad']) #ordeno

# creo df de provincias partidos y localidades
df_provincias = pd.DataFrame(df_places.provincia.unique(),columns=['nombre'])
df_partidos = pd.DataFrame(df_places.partido.unique(),columns=['nombre'])
df_localidades = pd.DataFrame(df_places.localidad.unique(),columns=['nombre'])

def buscar_reemplazar_place_column(row,column_name,df_base):
    if row[column_name]:
        idx = df_base.index[df_base.nombre == row[column_name]]
        return int(idx.data[0])
    
df_places.provincia =  df_places.apply(buscar_reemplazar_place_column,args=('provincia',df_provincias),axis=1)
df_places.partido =  df_places.apply(buscar_reemplazar_place_column,args=('partido',df_partidos),axis=1)
df_places.localidad =  df_places.apply(buscar_reemplazar_place_column,args=('localidad',df_localidades),axis=1)
df_places.sample(5)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  app.launch_new_instance()


,provincia,partido,localidad
place_with_parent_names,,,
|argentina|capital federal|monserrat|,5,150.0,NaN
|argentina|bs.as. g.b.a. zona sur|cañuelas|vicente casares|,2,23.0,365.0
|argentina|bs.as. g.b.a. zona sur|berazategui|ranelagh|,2,22.0,358.0
|argentina|buenos aires interior|tornquist|villa ventana|,4,122.0,558.0
|argentina|córdoba|villa del dique|,10,307.0,NaN


In [155]:
# agrega la relacion para las columnas que se vayan pasando respecto al dataframe de provincias, localidades y partidos

def agregar_columna_place(row,column_name,test):
    if (row.place_with_parent_names):
        return df_places.loc[row.place_with_parent_names][column_name]
    
provincias = df.apply(agregar_columna_place,args=('provincia','random'),axis=1)
df['provincia'] = pd.Series(provincias, index=df.index)
partidos = df.apply(agregar_columna_place,args=('partido','random'),axis=1)
df['partido'] = pd.Series(partidos, index=df.index)
localidades = df.apply(agregar_columna_place,args=('localidad','random'),axis=1)
df['localidad'] = pd.Series(localidades, index=df.index)
df.sample(3)

,level_0,index,Id,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,expenses,description,title,lat_geonames,lat-lon_geonames,lon_geonames,place_name_geonames,lat_mediaPorZona,lon_mediaPorZona,provincia,partido,localidad
115,115,116,116,sell,PH,Pinamar,|argentina|buenos aires costa atlántica|pinamar|,Argentina,Buenos Aires Costa Atlántica,3429971.0,NaN,NaN,NaN,175000.0,USD,3087787.5,175000.0,330.0,100.0,530.30303,1750.000000,NaN,NaN,"Excelente propiedad ubicada en Pinamar Norte a 50 metros del Mar.Chalet con entrada por dos calles!Se desarrolla toda en una planta. Tres dormitorios con placard. Dos baÃ±os y living comedor con vista al frente. Cocina completa con todos lo muebles y vista al jardin.Jardin totalmente perimetrico con cerco de alambre y plantas, cochera semi cubierta y espacio para otro auto. Parrilla.Alarma de movimiento y perimetral en todas sus aberturas.Gas natural y calefacciÃ³n en todos los ambientes.OPORTUNIDAD!","PH 330m² con Plantas, Buenos Aires Costa Atlántica, Pinamar, por U$S 175.000",-37.10795,"-37.10795,-56.8614",-56.86140,Pinamar,-37.109509,-56.860612,3.0,59.0,NaN
98173,98175,101920,101920,sell,apartment,Rosario,|argentina|santa fe|rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,66.0,NaN,NaN,NaN,NaN,"Brown 1843 -Rosario-Unidades de 1 y 2 dormitorios. Monoambientes. Piscina y Solarium. CocherasA metros de la costa, a pasos del Parque España, rodeado de verde y amplia recreación, se emplaza el edificio One Brown, con todas las comodidades de la construcción moderna y con todas las ventajas propias de una ubicación excelente.El proyecto se desarrolla en 14 pisos sobre calle Brown, entre Italia y Dorrego, con orientación hacia el Norte. Cuenta con 55 departamentos, divididos en 13 unidades de 2 dormitorios, 14 de 1 dormitorio, 11 de un dormitorio Plus (con vestidor) y 17 monoambientes. Ofrece además, 31 cocheras simples y 1 cochera doble, repartidas en planta baja y 2 niveles de subsuelo.Completan el emprendimiento una serie de amenidades en azotea, que incluyen piscina, solarium, terraza pergolada y asadores, mas un gimnasio en el primer piso del edificio.Encontrá todas nuestras propiedades en",Departamento - Centro,-33.16667,"-33.16667,-60.75",-60.75000,Rosario Department,-32.947316,-60.661852,24.0,506.0,NaN
17844,17845,18428,18428,sell,apartment,Caseros,|argentina|bs.as. g.b.a. zona oeste|tres de febrero|caseros|,Argentina,Bs.As. G.B.A. Zona Oeste,3862356.0,"-34.60597,-58.569059",-34.60597,-58.569059,145000.0,USD,2558452.5,145000.0,NaN,99.0,NaN,1464.646465,4.0,NaN,"Corredor Responsable: Marisol Espinosa - CMCPM 2736Contacto: Ariel Isern - MLS ID # 420631038-4En una ubicación privilegiada del barrio Caseros, sobre la esquina de GRAL. HORNOS y URQUIZA, a una tres cuadras de AV. SAN MARÍN con un intenso sector comercial, amplias vías de acceso y medios de trasporte, Posee un frente integral de vidrio, con recibidor y ascensor de frente. Ingresando hacia la Izq. encontramos living integrado al comedor culminando en una hermosa cocina muy iluminada, con balcón cerrado pudiendo ser utilizado de usos múltiples. Hacia la derecha encontramos lavadero, un hermoso balcón (equipado especialmente para niños con cerramientos de aluminio) y vidriado para apreciar hermosa vista hacia barrios (Ramos Mejía, Haedo, Morón) En la planta alta se encuentran 3 dormitorios y baño. PERMUTA: por casa en Mendoza (centro) ó MAr del Plata (sobre colón) Consultenos !!",INCREIBLE Dpto.4 Hamb. en DUPLEX Centro de Caseros,-36.59806,"-36.59806,-61.74306",-61.74306,Partido de Daireaux,-34.594365,-58.564128,1.0,19.0,310.0


In [156]:
df['state_name'].value_counts()

Capital Federal                 30685
Bs.As. G.B.A. Zona Norte        24763
Bs.As. G.B.A. Zona Sur          13556
Córdoba                         11582
Buenos Aires Costa Atlántica    9911 
Bs.As. G.B.A. Zona Oeste        9048 
Santa Fe                        9033 
Buenos Aires Interior           2251 
Río Negro                       792  
Mendoza                         656  
Tucumán                         635  
Neuquén                         587  
Corrientes                      575  
Misiones                        453  
Entre Ríos                      365  
Salta                           276  
Chubut                          259  
San Luis                        245  
La Pampa                        153  
Chaco                           57   
San Juan                        40   
Formosa                         32   
Tierra Del Fuego                31   
Catamarca                       27   
Jujuy                           26   
Santa Cruz                      20   
La Rioja    

In [157]:
df['provincia'].value_counts()

5.0     30685
0.0     24763
2.0     13556
10.0    11582
3.0     9911 
1.0     9048 
24.0    9033 
4.0     2251 
19.0    792  
16.0    656  
27.0    635  
18.0    587  
9.0     575  
17.0    453  
11.0    365  
20.0    276  
8.0     259  
22.0    245  
14.0    153  
7.0     57   
21.0    40   
12.0    32   
26.0    31   
6.0     27   
13.0    26   
23.0    20   
25.0    4    
15.0    4    
Name: provincia, dtype: int64

In [158]:
# Analisis del cambio de columnas realizado con place_with parent names

df.place_name = df.place_name[df.place_name.notnull()].map(str.lower)
df_place_name = df.place_name

df_place_name_not_in = df_place_name[~(df_place_name.isin(df_partidos.nombre))]
df_place_name_not_in_loc = df_place_name_not_in[~(df_place_name_not_in.isin(df_localidades.nombre))]
df_place_name_not_in_loc_prov = df_place_name_not_in_loc[~(df_place_name_not_in_loc.isin(df_provincias.nombre))]
df_place_name_not_in_loc_prov.unique()
#Puedo eliminar la columna place_name

array(['barrio el golf', 'buenos aires', 'barrio las glorietas',
       'nueva pompeya', 'barrio los lagos', 'enyoi', 'qbay yacht',
       'barrio la alameda', 'islas del canal', 'barrio los alisos',
       'barrioportezuelo', 'barrio la isla', 'barrio los tilos',
       'barrio los sauces', 'barrio cabos del lago',
       'barrio los castores', 'barrio barrancas del lago',
       'barrio el yacht'], dtype=object)

In [159]:
# Reemplazo los valores que encuentro en place_name y que estan definidos en partidos

df.place_name = df.place_name[df.place_name.notnull()].map(str.lower)

def buscar_reemplazar_place_definidos(row,column_name,df_base): # funcion que buscar y reemplaza de la columna base (provincia, localidad, partido)
    a = df_base.nombre.str.contains(row[column_name], regex=False).any()
    if a:
        idx = df_base.index[df_base.nombre == row[column_name]]
        return int(idx.data[0])
    
df_place_name = df.place_name
mask_in_partidos = (df_place_name.isin(df_partidos.nombre))
mask_not_column_partido = df.partido.isnull()

print("Partidos con null:" + str(df.partido.isnull().sum()))

Partidos con null:4403


In [160]:
df_reemplazar_part = df[mask_not_column_partido&mask_in_partidos].apply(buscar_reemplazar_place_definidos,args=('place_name',df_partidos),axis=1)
df.partido.update(df_reemplazar_part)
print("Partidos con null luego de procesar:" + str(df.partido.isnull().sum()))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  if __name__ == '__main__':


Partidos con null luego de procesar:1686


In [161]:
# Reemplazo los valores que encuentro en place_name y que estan definidos en localidades

df_place_name = df.place_name
mask_in_localidades = (df_place_name.isin(df_localidades.nombre))
mask_not_column_localidad = df.localidad.isnull()

print("Localidades con null: " + str(df.localidad.isna().sum()))

df_reemplazar_loc = df[mask_not_column_localidad&mask_in_localidades].apply(buscar_reemplazar_place_definidos,args=('place_name',df_localidades),axis=1)
df.localidad.update(df_reemplazar_loc)

print("Localidades con null luego de procesar: " + str(df.localidad.isna().sum()))

Localidades con null: 76915


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  if __name__ == '__main__':


Localidades con null luego de procesar: 65576


###### Ahora vamos a limpiar la columna surface

In [162]:
df_title = df[df.title.notnull()]
df_title.title = df_title.title.map(str.lower)
pattern_m2 = regex.compile("(\d+\s*) m2")

def get_m2(row):
    result = pattern_m2.search(row)
    try:
        str_aux = result.group(1)
        array_m2 = str_aux.split()
        m2 = array_m2[-1]
        try:
            m2 = float(m2)
            return m2
        except:
            return np.nan;
    except:
        return np.nan

print("Cantidad de nulos m2 total surface: ",df.surface_total_in_m2.isnull().sum())

m2_from_title = df_title.title.apply(get_m2)
m2_from_title[m2_from_title.notnull()]
df.surface_total_in_m2.update(m2_from_title)

print("Cantidad de nulos luego de procesar: ",df.surface_total_in_m2.isnull().sum())

Cantidad de nulos m2 total surface:  38360
Cantidad de nulos luego de procesar:  37906


In [163]:
#Vamos a generar los 39.328 datos faltantes en surface_total en la tabla, y lo vamos a hacer
#rellenando con el promedio del barrio de cada propiedad

print("Cantidad de nulos en surface_total_in_m2 antes:",df.surface_total_in_m2.isnull().sum())

# creamos una proporción de metros cubiertos sobre metros totales - no puede ser mayor a 1!
propcubierto = df.surface_covered_in_m2 / df.surface_total_in_m2
mask = propcubierto < 1
propcubierto_clean  = propcubierto[mask]

Cantidad de nulos en surface_total_in_m2 antes: 37906


In [164]:
#Relevamos algunos casos anómalos, donde la proporción da mayor a uno (inverosímil). Desestimamos estos 1200 registros para este procedimiento.
tempmask = propcubierto > 1
np.sum(tempmask)

1194

In [165]:
#Creamos una nueva variable sin esos datos anómalos
mask = propcubierto < 1
propcubierto_clean  = propcubierto[mask]

In [166]:
#Hay dos casos anómalos donde superficie cubierta es cero, los reemplazamos por np.nan para evitar conflictos en el siguiente paso.
masksurface0 = (df.surface_covered_in_m2 == 0)
df.surface_covered_in_m2[masksurface0].fillna(np.nan)

#Agregamos la columna al dataframe
df['propcubierto']=propcubierto_clean

#Agrupamos por provincia(partido) el porcentaje promedio de m2cubierto/m2total
avg_propcubiertobarrio = df.groupby('partido')["propcubierto"].mean().sort_values(ascending = False)

#Cantidad de datos para calcular la proporcion 
avg_propcubiertobarriocount = df.groupby('partido')["propcubierto"].count().sort_values(ascending = False)

#Condición de la regla 1
removerporcantidadmask = avg_propcubiertobarriocount > 30

#Cantidad de datos existentes en tabla
datos_en_tabla = df.groupby('partido')['partido'].count().sort_values(ascending = False)

#Divido los datos existentes sobre los datos totales y obtengo la relación para la regla DOS
proporcion = avg_propcubiertobarriocount / datos_en_tabla
proporcion.round(2).sort_values(ascending = True)

#Condición de la regla 2
removerporproporcionmask = proporcion > 0.25

#Genero máscara con ambas condiciones
proporcionmask2 = removerporcantidadmask & removerporproporcionmask
propvalidados = avg_propcubiertobarrio[proporcionmask2]

#Ahora iteramos por las propiedades que tienen el dato faltante de superficie total y les inputamos la proporción promedio
#del barrio al que pertenecen, usando como dato la superficie cubierta.
# SUPERFICIE TOTAL = (SUPERFICIE CUBIERTA / PROPORCION CUBIERTO TOTAL)

surface_total_in_m2_clean = []
for index, row in df.iterrows():
    
    if pd.isnull(row.surface_total_in_m2):    
        if(row.partido in propvalidados.index):        
            surface_total_in_m2_clean.append(row.surface_covered_in_m2 / propvalidados.loc[row.partido]) #VERSION CON PROPVALIDADOS NO CORRE
        else:
            surface_total_in_m2_clean.append(row.surface_total_in_m2)    
    else:
        surface_total_in_m2_clean.append(row.surface_total_in_m2)
df["surface_total_in_m2_nueva"] = surface_total_in_m2_clean
df["surface_total_in_m2_nueva"]

#dropeamos los casos NaN
surfacetotalnueva_condatos = df.surface_total_in_m2_nueva.dropna()

#y finalmente, los reemplazamos en la columna "surface_total_in_m2_nueva" original del DF
df.surface_total_in_m2.update(surfacetotalnueva_condatos)

print("Cantidad de nulos en surface_total_in_m2 despues:",df.surface_total_in_m2.isnull().sum())

Cantidad de nulos en surface_total_in_m2 despues: 20504


In [167]:
#eliminamos columnas auxiliares generadas
df = df.drop(labels=['provincia', 'partido', 'localidad', 'propcubierto', 'surface_total_in_m2_nueva'], axis= 1)

In [168]:
df.columns

Index(['level_0', 'index', 'Id', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms',
       'expenses', 'description', 'title', 'lat_geonames', 'lat-lon_geonames',
       'lon_geonames', 'place_name_geonames', 'lat_mediaPorZona',
       'lon_mediaPorZona'],
      dtype='object')

In [169]:
df.surface_total_in_m2.isnull().sum()

20504

# Precio

In [170]:
len(df[df['price'].isnull()])

17537

In [171]:
def completarValoresFaltantesEnFilaPrice(dataFrameRow):
    
    ##############################################################
    ##Actualizacion de Rooms utilizando el método definido en la notebbok auxiliar analize_description_title.ipynb
    updatedDataFameRow = updatePriceFromRowData(dataFrameRow)
        
    return updatedDataFameRow


In [172]:
 df = df.apply(completarValoresFaltantesEnFilaPrice, axis=1);

In [173]:
len(df[df['price'].isnull()])

13647

# Completar el valor del campo rooms

In [174]:
def completarValoresFaltantesEnFila(dataFrameRow):
    
    ##############################################################
    ##Actualizacion de Rooms utilizando el método definido en la notebbok auxiliar analize_description_title.ipynb
    updatedDataFameRow = updateRoomsFromRowData(dataFrameRow)
        
    return updatedDataFameRow

In [175]:
df = df.apply(completarValoresFaltantesEnFila, axis=1);

# Datos en las columnas description y title

In [176]:
print('Cantidad de registros description con valor nulo: ' + str(df.description.isna().sum()))
print('Cantidad de registros title con valor nulo: ' + str(df.title.isna().sum()))

Cantidad de registros description con valor nulo: 0
Cantidad de registros title con valor nulo: 0


# Columna price_usd_per_m2

In [177]:
#funcion que recibe una fila del data frame y un diccionario con las conversiones entre monedas locales y el USD
#este metodo intenta calcular el precio por m2 en USD a partir de otras columnas del data frame
def updatePriceUSDPerM2(dataFrameRow, dolarConversion):
    
    conversion = 1
    #si el dato currency no tiene valor, asumimos que los valores son en USD (conversion = 1)
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency) and dataFrameRow.currency != ''):
        conversion = dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_usd_per_m2)):
        #precio por m2 USD= precio total USD / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
        
        #precio por m2 USD = (precio total $ * conversion moneda) / superficie total        
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price * conversion) / dataFrameRow.surface_total_in_m2
    
        #precio por m2 USD = precio por m2 * conversion moneda
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_per_m2)):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_per_m2 * conversion
            
        #precio por m2 USD = precio aprox por m2 USD / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
            
        #precio por m2 USD = precio aprox por m2 local currency * conversion moenda / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price_aprox_local_currency * conversion) / dataFrameRow.surface_total_in_m2
            
    return dataFrameRow

In [178]:
#contamos price per m2 en USD antes de la ejecucion
df['price_usd_per_m2'].isna().sum()

49265

In [179]:
#aplicamos la funcion que actualiza los precios por m2 al data frame
df = df.apply(updatePriceUSDPerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [180]:
#verificamos la cantidad de vacíos luego de la ejecución
df['price_usd_per_m2'].isna().sum()

22069

###### Columna price_per_m2

In [181]:
#similar al calculo del m2 por USD, el calculo del m2 en moneda local usa otras columnas del data frame para obtener el valor
#recibe tambien un diccionario con la conversion de monedas locales a USD
def updatePricePerM2(dataFrameRow, dolarConversion):
            
    #si el dato currency no tiene valor, asumimos que el valor precio por m2 a guardar es en ARS
    conversion = 1 / dolarConversion['ARS'] #si el dolar esta a 65 ARS => el factor de conversión de USD a ARS es 1 / 65
    
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency)):
        conversion = 1 / dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_per_m2)):
        #precio por m2 = precio aprox total local currency / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_local_currency / dataFrameRow.surface_total_in_m2
        
        #precio por m2 = precio por m2 USD / conversion moneda
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_usd_per_m2)):
            dataFrameRow.price_per_m2 = dataFrameRow.price_usd_per_m2 / conversion
            
        #precio por m2 USD = (precio aprox por m2 USD / conversion) / superficie total
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_per_m2 = (dataFrameRow.price_aprox_usd / conversion) / dataFrameRow.surface_total_in_m2
             
    return dataFrameRow

In [182]:
#contamos price per m2 en USD antes de la ejecucion
df['price_per_m2'].isna().sum()

30421

In [183]:
#aplicamos la funcion que actualiza los precios por m2 al data frame
df = df.apply(updatePricePerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [184]:
#verificamos la cantidad de vacíos luego de la ejecución
df['price_per_m2'].isna().sum()

22069

# Agregamos algunos datos de amenities analizando las columnas description y title

##### Los métodos que aplican las regex sobre las columnas description y title están definidos en la notebook auxiliar analize_description_title

In [185]:
df['pileta'] = df.apply(lambda row: containsPool(row), axis = 1)
df['pileta'].value_counts()

0    101629
1    14437 
Name: pileta, dtype: int64

In [186]:
df['cochera'] = df.apply(lambda row: containsParking(row), axis = 1)
df['cochera'].value_counts()

0    74781
1    41285
Name: cochera, dtype: int64

In [187]:
df['balcon'] = df.apply(lambda row: containsBalcony(row), axis = 1)
df['balcon'].value_counts()

0    88648
1    27418
Name: balcon, dtype: int64

In [188]:
df['terraza'] = df.apply(lambda row: containsTerrace(row), axis = 1)
df['terraza'].value_counts()

0    89695
1    26371
Name: terraza, dtype: int64

In [189]:
df['parrilla'] = df.apply(lambda row: containsGrill(row), axis = 1)
df['parrilla'].value_counts()

0    100109
1    15957 
Name: parrilla, dtype: int64

# Ajustes de valores de las columnas

### En los casos en que la superficie cubierta sea mayor a la superficie total, se cambia el valor de la superficie cubierta por el de la total

In [195]:
# casos en los que sup cubierta > sup total
len(df[df['surface_total_in_m2'] < df['surface_covered_in_m2']])

1194

In [199]:
def ajustarValoresSuperficie(dataRow):
    if(not math.isnan(dataRow.surface_total_in_m2) and not math.isnan(dataRow.surface_covered_in_m2)):
        if(dataRow.surface_covered_in_m2 > dataRow.surface_total_in_m2):
            dataRow.surface_covered_in_m2 = dataRow.surface_total_in_m2
    return dataRow

In [200]:
df = df.apply(ajustarValoresSuperficie, axis=1)

In [201]:
# resultado luego de la correccion
len(df[df['surface_total_in_m2'] < df['surface_covered_in_m2']])

0

### En los casos de que la superficie total sea mayor a la cubierta, se agrega una nueva columna con el valor de la superficie no cubierta, que será la diferencia entre la superficie total y la superficie cubierta

In [202]:
# casos en la que la superficie cubierta no es el total de la superficie
len(df[df['surface_total_in_m2'] > df['surface_covered_in_m2']])

63871

In [203]:
df['surface_not_covered_in_m2'] = df['surface_total_in_m2'] - df['surface_covered_in_m2']

In [207]:
# resultado luego de la correccion 
len(df[df['surface_total_in_m2'] == (df['surface_covered_in_m2'] + df['surface_not_covered_in_m2'])])

88632

In [190]:
#Estado final del data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116066 entries, 0 to 116065
Data columns (total 36 columns):
level_0                       116066 non-null int64
index                         116066 non-null int64
Id                            116066 non-null int64
operation                     116066 non-null object
property_type                 116066 non-null object
place_name                    116066 non-null object
place_with_parent_names       116066 non-null object
country_name                  116066 non-null object
state_name                    116066 non-null object
geonames_id                   97893 non-null float64
lat-lon                       67784 non-null object
lat                           67784 non-null float64
lon                           67784 non-null float64
price                         102419 non-null float64
currency                      98528 non-null object
price_aprox_local_currency    98529 non-null float64
price_aprox_usd               98529 non-null float